In [ ]:
%loadFromPOM pom.xml

In [ ]:
import ca.uhn.fhir.context.FhirContext;
import ca.uhn.fhir.rest.client.api.*;
import ca.uhn.fhir.rest.api.MethodOutcome;
import org.hl7.fhir.r4.model.*;

In [ ]:
FhirContext ctx = FhirContext.forR4();
String serverBase = "http://localhost:8080/fhir";
IGenericClient client = ctx.newRestfulGenericClient(serverBase);

In [ ]:
// Find the practitioner we've choosen before. Her name is Kelly Smith
Bundle practitionerBundle = client.search().forResource(Practitioner.class)
    .where(Practitioner.NAME.matches().value("Kelly Smith"))
    .returnBundle(Bundle.class)
    .execute();

Practitioner practitioner = (Practitioner) practitionerBundle.getEntry().get(0).getResource();



In [ ]:
// find the schedule to use with practitioner
Bundle scheduleBundle = client.search().forResource(Schedule.class)
    .where(Schedule.ACTOR.hasId(practitioner.getIdElement().getIdPart()))
    .returnBundle(Bundle.class)
    .execute();
// get the first schedule
Schedule schedule = (Schedule) scheduleBundle.getEntry().get(0).getResource();

// list all actors  in the schedule
Reference locationRef = null;
for (Reference actor : schedule.getActor()) {
    System.out.println(actor.getReference());
    if (actor.getReference().contains("Location")) {
        locationRef = actor;
    }
}
// get the location Resource Object
Location location = client.read().resource(Location.class).withId(locationRef.getReferenceElement().getIdPart()).execute();

//print the location name and address
System.out.println(location.getName());
System.out.println(location.getAddress().getLine().get(0));
System.out.println(location.getAddress().getCity());
System.out.println(location.getAddress().getState());
System.out.println(location.getAddress().getPostalCode());


In [ ]:
// For the schedule found choose a time slot at 11 am on Monday 16th of september 2018
// find the slot
Bundle slotBundle = client.search().forResource(Slot.class)
    .where(Slot.SCHEDULE.hasId(schedule.getIdElement().getIdPart()))
    .where(Slot.START.exactly().day("2018-09-16"))
    .returnBundle(Bundle.class)
    .execute();
// Print the start time,end time and status for all slots
Slot slotFree = new Slot();
for (Bundle.BundleEntryComponent entry : slotBundle.getEntry()) {
    Slot slot = (Slot) entry.getResource();
    System.out.println("Start time: " + slot.getStart() + " End time: " + slot.getEnd() + " Status: " + slot.getStatus());
    if (slot.getStatus().equals(Slot.SlotStatus.FREE)) {
        slotFree = slot;
    }
}

In [ ]:
// Find a patient whose given name is 大明 and family name is 陳
Bundle results = client
    .search()
    .forResource(Patient.class)
    .where(Patient.GIVEN.matches().value("大明"))
    .and(Patient.FAMILY.matches().value("陳"))
    .returnBundle(Bundle.class)
    .execute();

// Get the first patient from the bundle
Patient patient = (Patient) results.getEntry().get(0).getResource();

In [ ]:
// Create an appointment
Appointment appointment = new Appointment();

// Set the status of the appointment
appointment.setStatus(Appointment.AppointmentStatus.BOOKED);

// Set the start time of the appointment
appointment.setStart(slotFree.getStart());

// Set the end time of the appointment
appointment.setEnd(slotFree.getEnd());

Reference refPatient = new Reference();
refPatient.setReference("Patient/" + patient.getIdElement().getIdPart());
refPatient.setDisplay(patient.getName().get(0).getGivenAsSingleString() + " " + patient.getName().get(0).getFamily());
appointment.addParticipant()
           .setActor(refPatient)
           .setStatus(Appointment.ParticipationStatus.ACCEPTED)
           .setRequired(Appointment.ParticipantRequired.REQUIRED);

Reference refPractitioner = new Reference();
refPractitioner.setReference("Practitioner/" + practitioner.getIdElement().getIdPart());
refPractitioner.setDisplay(practitioner.getName().get(0).getGivenAsSingleString() + " " + practitioner.getName().get(0).getFamily());
appointment.addParticipant()
           .setActor(refPractitioner)
           .setStatus(Appointment.ParticipationStatus.NEEDSACTION)
           .setRequired(Appointment.ParticipantRequired.REQUIRED);

Reference refLocation = new Reference();
refLocation.setReference("Location/" + location.getIdElement().getIdPart());
refLocation.setDisplay(location.getName());
appointment.addParticipant()
           .setActor(refLocation)
           .setStatus(Appointment.ParticipationStatus.NEEDSACTION)
           .setRequired(Appointment.ParticipantRequired.REQUIRED);

// Set the slot of the appointment
appointment.addSlot( new Reference("Slot/" + slotFree.getIdElement().getIdPart()));

// Set Status of the appointment
appointment.setStatus(Appointment.AppointmentStatus.PROPOSED);

// Print the appointment
// System.out.println(ctx.newJsonParser().setPrettyPrint(true).encodeResourceToString(appointment));

// Create the appointment on the server
MethodOutcome outcome = client.create().resource(appointment).execute();

// Print the id of the created appointment
System.out.println(outcome.getId().getValue());